<h1><center>Einführung in die Neuroinformatik - Übungsblatt 5</center></h1>
<h2><center>Dominik Authaler, Marco Deuscher und Carolin Schindler</center></h2>
<h2><center>Gruppe Q</center></h2>
<h2><center>Juni 2019</center></h2>

## Aufgabe 1: 

$$
\begin{array}{ccc}
\hline
\text{Schritt} & \text{Berechnung} & \text{Benötigte Variablen} & \text{Phase} \\
\hline
1 & u_1^{(1)} & x_1, w_{11}^{(1)}, x_2, w_{21}^{(1)}& \text{I}\\
2 & y_1^{(1)}& u_1^{(1)}, b_1^{(1)}& \text{I}\\
3 & u_1^{(2)}& y_1^{(1)}, w_{11}^{(2)}, y_2^{(1)}, w_{21}^{(2)}& \text{I}\\
4 & y_1^{(2)}& u_1^{(2)}, b_1^{(2)}& \text{I}\\
5 & \delta_1^{(2)}& y_1^{(2)}, T, u_1^{(2)}& \text{II}\\
6 & \delta_1^{(1)}& w_{11}^{(2)}, \delta_1^{(2)}, u_1^{(1)}& \text{III}\\
7 & w_{11}^{(1)}& x_1, x_2, \eta, \delta_1^{(1)} & \text{IV}\\
\hline
\end{array}
$$

## Aufgabe 2: Backpropagation selbst implementieren

In [1]:
%matplotlib widget
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)

### Funktionen

In [148]:
def tanh_deriv(inp):
    return 1 / (np.cosh(inp)**2)

def forward(inpX, w1, w2, b1, b2):
    '''
    Perform a forward step of the network. For the transfer function in the hidden layer, use tanh.
    
    Parameters
    ----------
    inpX : data matrix
        input matrix, shaped as: samples x dimensions
    w1 : matrix
        weight matrix between input and hidden neurons
    w2 : matrix
        weight matrix between hidden and output neurons
    b1 : vector
        bias vector for the hidden neurons
    b2 : vector
        bias vector for the output neurons
    '''
    
    u1 = np.dot(inpX.transpose(), w1)
    y1 = np.tanh(np.add(u1, b1))
    y2 = np.asscalar(np.add(np.dot(y1, w2), b2))
    
    #print(inpX.transpose().shape)
    #print(w1.shape)
    #print(w2.shape)
    #print(b1.shape)
    #print(b2.shape)
    
    #print("Res: ")
    #print(y1.shape)
    
    #print(y2)
    
    return u1, y1, y2

def initialize_weights(inpDim, hiddenNeurons, outDim):
    '''
    Initialize the weight matrix based on input Dimension, amount of hidden neurons and output dimension.
    The range for the initial weights is given by [-.5; .5].
    
    Parameters
    ----------
    inpDim : int
        Number of input neurons
    hiddenNeurons : int
        Number of hidden neurons
    outDim : int
        Number of output neurons
    weights : list
        List containing the weights and biases in the following order: [w1, w2, b1, b2]
    '''
    
    w1 = np.subtract(np.random.rand(inpDim, hiddenNeurons), 0.5)
    
    w2 = np.subtract(np.random.rand(hiddenNeurons, outDim), 0.5)
    
    b1 = np.subtract(np.random.rand(1, hiddenNeurons), 0.5)
    
    b2 = np.subtract(np.random.rand(1, outDim), 0.5)
    
    #print(w1.shape)
    #print(w2.shape)
    #print(b1.shape)
    #print(b2.shape)
    
    weights = [w1, w2, b1, b2];
    
    return weights

def prop_error(T, y2, w2, transDiff_u1):
    '''
    Calculation of the error of the network
    
    Parameters
    ----------
    T : float
        teaching signal of the current sample
    y2 : float
        output of the last neuron
    w2 : data matrix
        weight matrix between hidden and output layer
    transDiff_u1 : vector
        differential of the transfer function used on u1
    '''
    
    # Ableitung von Transferfunktion f(x) = x ist 1
    delta2 = (T-y2);
    
    delta1 = w2.transpose()*delta2*transDiff_u1
    
    return delta1, delta2

def training(hiddenNeurons, lernRate, inpX, outT, epoch):
    '''
    Train the neural network. 
    
    Parameters
    ----------
    hiddenNeurons : int
        Number of hidden Neurons
    lernRate : float
        Lernrate \eta
    inpX : data matrix
        input data and shaped as: samples x dimensions 
    outT : vector
        teaching signal: one dimensional vector
    epoch : int
        number of training epochs
    '''
    
    [outDim, length_T] = outT.shape
    [inpDim, length_I] = inpX.shape
    
    assert length_T == length_I, "Sizes doesn't fit!"
    
    print('Training des Netzes mit ' + str(length_T) + " Lehrersignalen")
    
    [w1, w2, b1, b2] = initialize_weights(inpDim, hiddenNeurons, outDim)
    errorList = []
    
    for e in range (1, epoch):
        inpPerm = np.random.permutation(inpX)
        for i in range (0, length_T-1):
        #for i in range (0, 1):
            samplePoint = inpPerm[:,[i]]
            #samplePoint = inpX[:, [i]]
            T = outT[:,i]

            [u1, y1, y2] = forward(samplePoint, w1, w2, b1, b2)
            [d1, d2] = prop_error(T, y2, w2, tanh_deriv(u1))

            #print("y1: " + str(y1.shape))
            #print("w1: " + str(w1.shape))
            #print("d1: " + str(d1.shape))
            #print("d2: " + str(d2.shape))
            #print("x: " + str(samplePoint.shape))
            #print("b1: " + str(b1.shape))
            #print("b2: " + str(b2.shape))

            #Lernschritt
            w2 = np.add(w2, lernRate*y1.transpose()*d2)
            w1 = np.add(w1, lernRate*samplePoint*d1)

            b2 = b2 + lernRate*d2
            b1 = b1 + lernRate*d1


        ## Netzwerkfehler berechnen
        error = 0
        for i in range (0, length_T-1):
            samplePoint = inpX[:, [i]]
            T = np.asscalar(outT[:,i])

            [u1, y1, y2] = forward(samplePoint, w1, w2, b1, b2)

            #print("T: " + str(T))
            #print("Y2: " + str(y2))

            error += (T-y2)**2

        #print("ERROR: " + str(error))
        errorList.append(error)
    return errorList

### Initialiserung der Parameter und Training:
- X und Y entsprechen dem Datensatz
- Z ist das Lehrersignal

In [150]:
############
## Generate some sample data
def f(x,y):
    return np.sin(np.sqrt(x**2 + y**2)) + np.cos(.9*(x-y))

X = np.linspace(-6, 6, 30)
Y = np.linspace(-6, 6, 30)
x, y = np.meshgrid(X, Y)
z = f(x, y)

# Change the shape of the data points
outT = np.reshape(z, (1, 900))
inpX = np.reshape(np.array([x, y]), (2, 900))
#print(outT.shape)
#print(inpX.shape)

############
## Initialize network parameter
inputNeuronen = 2
hiddenNeuronen = 100
lernRate       = 0.0001
epochen        = 1000
outputNeuronen = 1;

#############
## Start training of the neural network
errors = training(hiddenNeuronen, lernRate, inpX, outT, epochen)

print("Fehlerentwicklung: " + str(errors))

Training des Netzes mit 900 Lehrersignalen
Fehlerentwicklung: [1047.9781269758844, 883.0670481408075, 809.9795675021808, 774.411250442277, 755.3607411181522, 743.7894567138237, 735.7411667667105, 723.7377661622774, 721.4620254201225, 717.6932344926629, 709.0087403544437, 704.2469444745426, 704.1158646537439, 700.8715574381503, 692.5432844966878, 688.4762233652383, 684.5899538138877, 685.9434599024481, 677.2137387906812, 679.2586442531834, 670.245603769631, 672.6927288531018, 663.4005029287455, 666.2344580889924, 656.6783302477246, 653.1687361481889, 656.4962233761398, 653.5060856567096, 650.5013112619675, 640.544993838532, 637.0781822770743, 633.7687719218675, 630.6387775978014, 627.5902439048537, 632.1030905505592, 629.0651089005014, 618.4635759032911, 615.4997285320538, 620.467582135007, 609.6997093808715, 615.1400623000258, 612.4930729847521, 609.8116382702291, 607.2401818364133, 604.7039952989131, 593.94082938451, 599.0739872542481, 588.4843305221297, 594.1185779106966, 583.3326911

### Visualisierung der Daten

In [154]:
fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')
surf1 = ax1.plot_surface(x,y,z, cmap = 'plasma',
                       linewidth=0, antialiased=False)
ax1.set_title('Original')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('Original');

fig2 = plt.figure()
ax2 = fig2.add_subplot(111, projection='3d')
surf2 = ax2.plot_surface(x,y,z, cmap = 'plasma',
                       linewidth=0, antialiased=False)
ax2.set_title('Approximation')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('Approximation');

FigureCanvasNbAgg()

FigureCanvasNbAgg()